# S3 based distributive lock implementation

It is a S3 based distributive lock implementation.

For example, we have many workers working on the same task, but we want to ensure that only one worker can work on this task at one time. Worker should create an S3 object to claim the ownership of the lock before doing the task. If the lock is already exists, then don't do any task. When the owner of the lock finishes the task, then release the lock by update the content of the S3 object.

In [1]:
from boto_session_manager import BotoSesManager
from fixa.aws.aws_s3_lock import Lock, Vault, get_utc_now, AlreadyLockedError

In [2]:
bsm = BotoSesManager(profile_name="awshsh_app_dev_us_east_1", region_name="us-east-1")
bucket = f"{bsm.aws_account_id}-{bsm.aws_region}-data"
s3_client = bsm.s3_client

In [5]:
# define a vault backend
vault = Vault(
    bucket=bucket,
    key="projects/aws_s3_lock/my-task.json",
    expire=900, # how long we automatically release a dead lock
    wait=1.0, # the higher the value, the more reliable of this mechanism but we need to wait longer before doing any task
)

In [9]:
# acquire the lock before doing any task
lock = vault.acquire(s3_client, owner="alice")
lock

Lock(expire=900, lock_time='2023-09-10T18:39:06.066075+00:00', release_time=None, owner='alice')

In [10]:
# this will fail because the lock is already owned by alice
bob_lock = vault.acquire(s3_client, owner="bob")

AlreadyLockedError: Lock is already acquired by Lock(expire=900, lock_time='2023-09-10T18:39:06.066075+00:00', release_time=None, owner='alice')

In [11]:
# alice can still acquire the lock
alice_lock = vault.acquire(s3_client, owner="alice")
alice_lock

Lock(expire=900, lock_time='2023-09-10T18:39:26.197306+00:00', release_time=None, owner='alice')

In [12]:
# alice do some task
...

# alice release the lock after the task is done
vault.release(s3_client, lock)

Lock(expire=900, lock_time='2023-09-10T18:39:06.066075+00:00', release_time='2023-09-10T18:39:52.458908+00:00', owner=None)

In [13]:
# now bob can acquire the lock
lock = vault.acquire(s3_client, owner="bob")
lock

Lock(expire=900, lock_time='2023-09-10T18:40:09.953411+00:00', release_time=None, owner='bob')